In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import pandas as pd 
import numpy as np
#import cPickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
#from keras.engine.topology import Layer, InputSpec
from keras import initializers
from keras.utils.layer_utils import get_source_inputs


In [ ]:
df_HARD = pd.read_csv("balanced-reviews.txt", sep="\t", header=0,encoding='utf-16')

DATA_COLUMN = "text"
LABEL_COLUMN = "label"

df_HARD = df_HARD[["review","rating"]]  # we are interested in rating and review only
df_HARD.columns = [DATA_COLUMN, LABEL_COLUMN]
print(df_HARD[LABEL_COLUMN].value_counts())
# code rating as +ve if > 3, -ve if less, no 3s in dataset

In [ ]:
df_HARD

In [ ]:
df_HARD2 = pd.read_csv("balanced-reviews.txt", sep="\t", header=0,encoding='utf-16')

df_HARD2 = df_HARD2[["review","rating"]]  # we are interested in rating and review only
df_HARD2.columns = [DATA_COLUMN, LABEL_COLUMN]

# code rating as +ve if > 3, -ve if less, no 3s in dataset

hard_map = {
    5: 'POS',
    4: 'POS',
    2: 'NEG',
    1: 'NEG'
}

df_HARD2[LABEL_COLUMN] = df_HARD2[LABEL_COLUMN].apply(lambda x: hard_map[x])

print(df_HARD2[LABEL_COLUMN].value_counts())



In [ ]:
df_HARD.columns

In [ ]:
import re
def deNoise(text):
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ،    | #commas
                             ؛    | # commas
                             \*    | #stars
                             \.   | # full stop
                             \( | \) | #parenthesis
                             ,,     | #double commas
                             ,      | #single comma
                             ;       |#sim col
                             :    | #col
                             _    | #underscore 
                             \=  | #eq
                             \\\ | #comm
                             \/   | #comme
                              -   | #dash
                              ـ     # Tatwil/Kashida
                              
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub("[إأٱآا]", "ا", text)
    return text
def normalizeArabic(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("[https]", ' ', text)
    return(text)

In [ ]:
df_HARD[DATA_COLUMN]=[deNoise(i)for i in df_HARD[DATA_COLUMN]] 
df_HARD[DATA_COLUMN]=[normalizeArabic(i)for i in df_HARD[DATA_COLUMN]]  
    

In [ ]:
df_HARD2[DATA_COLUMN]=[deNoise(i)for i in df_HARD2[DATA_COLUMN]] 
df_HARD2[DATA_COLUMN]=[normalizeArabic(i)for i in df_HARD2[DATA_COLUMN]]

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df_HARD, test_size=0.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
train2,test2 = train_test_split(df_HARD2, test_size=0.20, random_state=42)

In [ ]:
TOTAL_TRAIN= list(train['text'])+ list(train2['text'])

In [ ]:
TOTAL_TEST= list(test['text'])+ list(test2['text'])

In [ ]:
token= Tokenizer(filters=('!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'))
token.fit_on_texts(list(TOTAL_TRAIN)  + list(TOTAL_TEST))
#tokenizer.fit_on_texts(list(train_data['text']) + list(test_data['text']))
vocab= len(token.word_index)+1

In [ ]:
vocab

In [ ]:
max_len=50
train['text'] =token.texts_to_sequences(train['text'])
test['text'] =token.texts_to_sequences(test['text'])

train2['text'] =token.texts_to_sequences(train2['text'])
test2['text'] =token.texts_to_sequences(test2['text'])


In [ ]:
x_train = pad_sequences(list(train['text']), maxlen=max_len, padding='post', truncating='post')
x_test = pad_sequences(list(test['text']), maxlen=max_len, padding='post', truncating='post')
#------
x_train2 = pad_sequences(list(train2['text']), maxlen=max_len, padding='post', truncating='post')
x_test2 = pad_sequences(list(test2['text']), maxlen=max_len, padding='post', truncating='post')

In [ ]:
import kutilities # to install this module use :  pip install keras-utilities
import numpy as np


In [ ]:
from kutilities.helpers.data_preparation import  categories_to_onehot,labels_to_categories, get_labels_to_categories_map, get_class_weights2, onehot_to_categories
#TASK4 class
y_train= labels_to_categories(train['label'])
y_test= labels_to_categories(test['label'])
y_train=categories_to_onehot(y_train)
y_test=categories_to_onehot(y_test)

In [ ]:
y_train2= labels_to_categories(train2['label'])
y_test2= labels_to_categories(test2['label'])
y_train2=categories_to_onehot(y_train2)
y_test2=categories_to_onehot(y_test2)

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
from TransformerLayers import *

In [ ]:
class SentenceEncoder(tf.keras.layers.Layer):
    def __init__(self,vocab_size,nb_layers,hidden_dim,filter_size,num_heads,dropout,maximum_position_encoding=200):
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.dropout = dropout
        self.filter_size= filter_size
        self.nb_layers = nb_layers
        self.vocab_size = vocab_size
        self.maximum_position_encoding = maximum_position_encoding
        super(SentenceEncoder,self).__init__()
    def build(self,input_shape):
        self.embedding_layer = tf.keras.layers.Embedding(self.vocab_size, 
                                        self.hidden_dim,
                                        )
        self.encoder = EncoderSubnetwork(nb_layers=self.nb_layers,
                                         hidden_dim=self.hidden_dim,
                                         filter_size=self.filter_size,
                                         num_heads=self.num_heads,dropout=self.dropout) 
        self.outputPool = tf.keras.layers.GlobalAveragePooling1D()
        self.pos_encoding = positional_encoding(self.maximum_position_encoding,
                                            self.hidden_dim)

        super(SentenceEncoder,self).build(input_shape)
    def call(self,input_x,training=False,return_attention_weights=False):
        src_embedding = self.embedding_layer(input_x)
        seq_len = tf.shape(input_x)[1]
        src_embedding *= tf.math.sqrt(tf.cast(self.hidden_dim, tf.float32))
        src_embedding += self.pos_encoding[:, :seq_len, :]


        attention_bias = get_padding_bias(input_x)
        
        sentence_encode = self.encoder(src_embedding,
                                       attention_bias,
                                       training,
                                       return_attention_weights=return_attention_weights)
        if return_attention_weights:
            sentence_encode, attention_weights = sentence_encode
        sentence_encode = self.outputPool(sentence_encode)
        return sentence_encode if not return_attention_weights else [sentence_encode, attention_weights]

In [ ]:
src_input = tf.keras.layers.Input(shape=(None,))
src_input2 = tf.keras.layers.Input(shape=(None,))

In [ ]:
sentence_encoder = SentenceEncoder(vocab,nb_layers=7,hidden_dim=128,filter_size=256,num_heads=4,dropout=0.1)


In [ ]:
sentence_embedding =sentence_encoder(src_input)
sentence_embedding2 =sentence_encoder(src_input2)

In [ ]:
dense_layer_1 = tf.keras.layers.Dense(4, activation='softmax')(sentence_embedding)
dense_layer_2 = tf.keras.layers.Dense(2, activation='sigmoid')(sentence_embedding2)

In [ ]:
model = tf.keras.models.Model(inputs=src_input, outputs=dense_layer_1)
model2 = tf.keras.models.Model(inputs=src_input2, outputs=dense_layer_2)

In [ ]:
model2.summary()

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
learning_rate = CustomSchedule(128)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
model2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])

In [ ]:
for x in range(8):
    # Check if x is even
    if x % 2 == 0:
        print('4class')
        history= model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=1, validation_data=(x_test,y_test))

    else:
        print('2class')
        history_2=model2.fit(x_train2, y_train2, batch_size=64, epochs=1, verbose=1, validation_data=(x_test2,y_test2)) 

In [ ]:
print("f1:",metrics.f1_score(y_pre,y_test,average='weighted'))

In [ ]:
y_test=categories_to_onehot(y_test)